In [ ]:
import numpy as np

In [ ]:
def gas_appl_chem_pot_calc(T,P,size):
    #define universal constants
    plank = 6.62607004e-34 #J*s
    kB = 1.38064852E-23 #J/K
    kB2 = 8.6173303e-5 #eV/K
    pi = np.pi

    #define constants realted to H2
    red_mH2 = 8.36868e-28 #kg
    tot_mH2 = 3.34767e-27 #kg
    bond_H2 = 7.4973e-11 #m
    I_H2 = red_mH2*bond_H2**2
    v_H2 = 0.536646093 #eV
    Tc_H2 = 33.19 # K
    Pc_H2 = 13.13 # bar
    Pc_H2 = Pc_H2*10**5 #Pa
    w_H2 = -0.216
    B0_H2 = 0.083 - 0.422/(T/Tc_H2)**1.6
    B1_H2 = 0.139 - 0.172/(T/Tc_H2)**4.2

    #define constants related to H2O
    tot_mH2O = 2.99158E-26 #kg
    I_H2O = 6.289E-141 #kg^3*m^6
    v_H2O = np.array([0.476270676, 0.461783424, 0.195599118]) #eV
    Tc_H2O = 647.1 # K
    Pc_H2O = 220.55 # bar
    Pc_H2O = Pc_H2O*10**5 #Pa
    w_H2O = 0.345
    B0_H2O = 0.083 - 0.422/(T/Tc_H2O)**1.6
    B1_H2O = 0.139 - 0.172/(T/Tc_H2O)**4.2

    #calculate pressure and temperature dependent chemical potentials
    ChemPotH2 = np.array([])
    ChemPotH2O = np.array([])
    for p_H2 in P:
        for p_H2O in P:
            #H2 gas phase partition functions
            qtrans_H2 = (2*pi*tot_mH2*kB*T)**(3/2)/plank**3*kB*T/p_H2 
            qrot_H2 = 8*pi**2*I_H2*kB*T/(2*plank**2)
            hold1 = np.exp(-v_H2/kB2/T)
            hold2 = np.exp(-v_H2/kB2/T/2)
            qvib_H2 = hold2/(1-hold1)

            #H2O gas phase partition functions
            qtrans_H2O = (2*pi*tot_mH2O*kB*T)**(3/2)/plank**3*kB*T/p_H2O 
            qrot_H2O = 8*pi**2/(2*plank**3)*(2*pi*kB*T)**(3/2)*(I_H2O)**(1/2)
            qvib = np.array([])
            for pnts in v_H2O:
                hold3 = np.exp(-pnts/kB2/T)
                hold4 = np.exp(-pnts/kB2/T/2)
                hold5 = hold4/(1-hold3)
                qvib = np.append(qvib, hold5)
            qvib_H2O = np.prod(qvib)

            #Gibbs residuals based on 2nd virial coefficient correlation
            resid_H2 = kB2*T*(p_H2/Pc_H2)*(T/Tc_H2)**(-1)*(B0_H2 + w_H2*B1_H2)
            resid_H2O = kB2*T*(p_H2O/Pc_H2O)*(T/Tc_H2O)**(-1)*(B0_H2O + w_H2O*B1_H2O)

            #gas phase chemical potential
            part_H2 = qtrans_H2*qrot_H2*qvib_H2
            part_H2O = qtrans_H2O*qrot_H2O*qvib_H2O
            temp_H2 = kB2*T*np.log(1/part_H2) + resid_H2
            temp_H2O = kB2*T*np.log(1/part_H2O) + resid_H2O
            ChemPotH2 = np.append(ChemPotH2, temp_H2)
            ChemPotH2O = np.append(ChemPotH2O, temp_H2O)

    ChemPotO = ChemPotH2O-ChemPotH2
    ChemPotH = ChemPotH2/2

    ChemPotH = np.reshape(ChemPotH,(size,size))
    ChemPotO = np.reshape(ChemPotO,(size,size))
    np.savetxt('ChemPotH_'+str(T)+'K.txt',ChemPotH,fmt='%s')
    np.savetxt('ChemPotO_'+str(T)+'K.txt',ChemPotO,fmt='%s')

In [ ]:
#define pressure and temperature
sizeP = 200
P = np.linspace(-12,5,sizeP)
P = 10**P # bar
P = P*10**5 # Pa
T = [200,500,1000,1500,2000] # K

for item in T:
    gas_appl_chem_pot_calc(item,P,sizeP)
    print(str(item)+'K completed!')